In [17]:
%%bash
rm -rf /home/jordan990301/PCA_Experiments/data/outputs/juicer
mkdir /home/jordan990301/PCA_Experiments/data/outputs/juicer

HIC_PATH="/home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic"
OUTPUT_PATH="/home/jordan990301/PCA_Experiments/data/outputs/juicer"
JUICER_TOOLS_PATH="/home/jordan990301/PCA_Experiments/juicer/juicer_tools.jar"

java -jar $JUICER_TOOLS_PATH pearsons KR $HIC_PATH 5 BP 1000000 $OUTPUT_PATH/pearson_matrix.txt
java -jar $JUICER_TOOLS_PATH eigenvector KR $HIC_PATH 5 BP 1000000 $OUTPUT_PATH/eigenvector.txt

WARN [2023-11-06T16:27:20,942]  [Globals.java:138] [main]  Development mode is enabled
Reading file: /home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic
WARN [2023-11-06T16:27:21,677]  [Globals.java:138] [main]  Development mode is enabled


In [18]:
import sys
import numpy as np
import pandas as pd
import hicstraw
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
hic_path = "/home/jordan990301/PCA_Experiments/data/Rao2014/samples/GSM1551550_HIC001.hic"
output_path = "/home/jordan990301/PCA_Experiments/data/outputs/juicer"
chrom_name_x = "5"
chrom_name_y = "5"
chrom_type = "BP"
norm = "KR"
resolution = 1000000

In [20]:
hic = hicstraw.HiCFile(hic_path)

In [21]:
print(hic.getGenomeID())
print(hic.getResolutions())

hg19
[2500000, 1000000, 500000, 250000, 100000, 50000, 25000, 10000, 5000]


In [22]:
selected_chromosome_size = None

for chrom in hic.getChromosomes():
    print(chrom.name, chrom.length)
    if (chrom.name == chrom_name_x):
        selected_chromosome_size = int(chrom.length)

print(f"\n{selected_chromosome_size}")

All 3098789
1 249250621
2 243199373
3 198022430
4 191154276
5 180915260
6 171115067
7 159138663
8 146364022
9 141213431
10 135534747
11 135006516
12 133851895
13 115169878
14 107349540
15 102531392
16 90354753
17 81195210
18 78077248
19 59128983
20 63025520
21 48129895
22 51304566
X 155270560
Y 59373566
MT 16569

180915260


In [23]:
matrix_oe = hic.getMatrixZoomData(chrom_name_x, chrom_name_y, "oe", norm, chrom_type, resolution)
numpy_matrix_oe = matrix_oe.getRecordsAsMatrix(0, selected_chromosome_size, 0, selected_chromosome_size)

In [24]:
oe_matrix_df = pd.DataFrame(numpy_matrix_oe)
display(oe_matrix_df)

np.savetxt(f'{output_path}/oe_matrix.txt', oe_matrix_df.values, fmt='%1.4e')

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,180
0,1.428854,1.583371,0.566957,0.633202,0.769255,1.414538,1.805802,1.125693,0.876512,1.231929,...,1.136701,1.738887,1.421194,1.258349,2.320294,2.971284,3.746220,2.424965,5.538469,8.357439
1,1.583371,1.229068,0.671415,0.789489,1.211582,1.869440,2.212331,1.428301,1.024145,1.310579,...,1.164590,1.083811,1.131598,0.785750,1.650655,1.933119,2.045736,2.064830,2.501890,2.770853
2,0.566957,0.671415,0.943425,1.518452,3.050152,1.838924,1.293288,1.779932,1.734125,1.407206,...,0.439937,0.274506,0.518259,0.967340,0.527815,0.469486,1.265151,3.057574,1.542840,0.992367
3,0.633202,0.789489,1.518452,0.938129,1.419984,0.988762,0.901753,1.453098,1.430640,1.175734,...,0.227104,0.191352,0.491704,1.052708,0.510840,0.191783,1.116517,2.427305,0.808373,0.755266
4,0.769255,1.211582,3.050152,1.419984,0.883754,0.736239,0.815878,1.432495,1.420572,1.359838,...,0.183334,0.328765,0.362153,0.916054,0.371194,0.334629,0.593846,1.939191,0.492770,0.509582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2.971284,1.933119,0.469486,0.191783,0.334629,0.693630,0.831873,0.405116,0.116047,0.680688,...,1.182340,1.253627,0.871726,0.605144,1.069735,1.457853,0.795832,0.413847,1.287612,1.131768
177,3.746220,2.045736,1.265151,1.116517,0.593846,1.278956,1.288432,0.850730,0.675695,0.794992,...,1.015213,1.147524,0.980961,1.161808,2.327762,0.795832,1.106407,1.417468,2.205024,1.577548
178,2.424965,2.064830,3.057574,2.427305,1.939191,1.170152,0.822033,1.204244,0.913267,0.660916,...,0.801593,0.951589,0.987066,1.388159,1.110867,0.413847,1.417468,1.276572,1.052683,1.086581
179,5.538469,2.501890,1.542840,0.808373,0.492770,0.944026,0.994947,0.698778,0.595976,1.081011,...,1.320220,1.403293,1.215181,1.464436,1.607257,1.287612,2.205024,1.052683,1.206720,1.169880


In [25]:
corr_matrix_df = oe_matrix_df.corr(method='pearson')
display(corr_matrix_df)

np.savetxt(f'{output_path}/corr_matrix.txt', corr_matrix_df.values, fmt='%1.4e')

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,180
0,1.000000,0.766796,0.152874,0.041194,-0.005135,0.308545,0.385224,0.101435,0.010875,0.223020,...,0.404159,0.415949,0.262858,0.139480,0.433873,0.454075,0.496431,0.088680,0.372653,0.290567
1,0.766796,1.000000,0.373647,0.231895,0.194346,0.599224,0.721890,0.412960,0.276114,0.538281,...,0.333496,0.341430,0.281849,0.146241,0.380229,0.403136,0.488841,0.150702,0.459160,0.304066
2,0.152874,0.373647,1.000000,0.932446,0.904483,0.756157,0.517070,0.782794,0.792754,0.714698,...,-0.369733,-0.325069,-0.321462,-0.081688,-0.261422,-0.331439,0.034656,0.421448,-0.103490,-0.138453
3,0.041194,0.231895,0.932446,1.000000,0.942842,0.692239,0.432008,0.709440,0.744838,0.625110,...,-0.387791,-0.376485,-0.315651,-0.138463,-0.303095,-0.374152,-0.027352,0.407179,-0.113775,-0.165386
4,-0.005135,0.194346,0.904483,0.942842,1.000000,0.748308,0.485951,0.795013,0.836960,0.682171,...,-0.450065,-0.429465,-0.359520,-0.164835,-0.360218,-0.393147,-0.041225,0.419316,-0.125556,-0.163678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,0.454075,0.403136,-0.331439,-0.374152,-0.393147,-0.100552,0.174119,-0.292432,-0.390524,-0.188952,...,0.829457,0.841253,0.655340,0.303181,0.782703,1.000000,0.674515,0.050911,0.762192,0.671329
177,0.496431,0.488841,0.034656,-0.027352,-0.041225,0.195639,0.338328,0.000562,-0.075183,0.116094,...,0.544124,0.618554,0.456676,0.423196,0.675645,0.674515,1.000000,0.515467,0.815863,0.748755
178,0.088680,0.150702,0.421448,0.407179,0.419316,0.319303,0.242005,0.258212,0.278980,0.258907,...,-0.018456,0.023168,0.030721,0.174594,0.166721,0.050911,0.515467,1.000000,0.394609,0.275358
179,0.372653,0.459160,-0.103490,-0.113775,-0.125556,0.135595,0.375494,-0.049480,-0.135673,0.032987,...,0.589944,0.616614,0.596263,0.294543,0.690713,0.762192,0.815863,0.394609,1.000000,0.832764


In [26]:
# pd.read_table(f"{output_path}/oe_matrix.txt", header=None, sep=" ")

In [27]:
### Calculate from juicer_tools
pearson_df = pd.read_csv(f"{output_path}/pearson_matrix.txt", header=None, sep=" ")
pearson_df.pop(pearson_df.columns[-1])
pearson_df

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,180
0,1.000000,0.780338,0.242945,0.135347,0.083897,0.367733,0.442912,0.170841,0.090397,0.284370,...,0.377778,0.402220,0.268889,0.047628,0.412883,0.500784,0.538462,0.099710,0.430239,0.337633
1,0.780338,1.000000,0.490740,0.359782,0.310631,0.674346,0.752071,0.495227,0.370468,0.601034,...,0.240381,0.283286,0.152579,0.024185,0.289431,0.382688,0.465376,0.141032,0.424695,0.312102
2,0.242945,0.490740,1.000000,0.937640,0.899608,0.783062,0.598955,0.834079,0.854563,0.777822,...,-0.395836,-0.333316,-0.365968,-0.150578,-0.259977,-0.259169,0.116928,0.395593,-0.096052,-0.093432
3,0.135347,0.359782,0.937640,1.000000,0.948982,0.756972,0.535917,0.811995,0.868709,0.737725,...,-0.450025,-0.399580,-0.424750,-0.206969,-0.323965,-0.324431,0.057475,0.365411,-0.150044,-0.118222
4,0.083897,0.310631,0.899608,0.948982,1.000000,0.768472,0.549000,0.842487,0.896083,0.738823,...,-0.483112,-0.433402,-0.450946,-0.236325,-0.364260,-0.323539,0.041959,0.401898,-0.145627,-0.116895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,0.500784,0.382688,-0.259169,-0.324431,-0.323539,-0.072001,0.157531,-0.261143,-0.329997,-0.175482,...,0.831442,0.846813,0.719199,0.254875,0.776324,1.000000,0.664738,0.044228,0.787498,0.676562
177,0.538462,0.465376,0.116928,0.057475,0.041959,0.180225,0.283901,0.002702,-0.037798,0.090763,...,0.485872,0.573624,0.445353,0.280986,0.613293,0.664738,1.000000,0.519924,0.796414,0.695573
178,0.099710,0.141032,0.395593,0.365411,0.401898,0.277750,0.206563,0.237989,0.288169,0.233057,...,-0.089827,-0.024784,-0.042317,0.056476,0.103726,0.044228,0.519924,1.000000,0.355084,0.255423
179,0.430239,0.424695,-0.096052,-0.150044,-0.145627,0.120386,0.302241,-0.085427,-0.173523,-0.027694,...,0.631774,0.694191,0.582963,0.275968,0.693200,0.787498,0.796414,0.355084,1.000000,0.832573


In [28]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imp.fit(corr_matrix_df)
corr_matrix_df = imp.transform(corr_matrix_df)

In [29]:
# corr_matrix_df doesn't exist NaN
print(np.isnan(corr_matrix_df).any())
print(len(corr_matrix_df[0]))
print(len(corr_matrix_df[1]))

False
181
181


In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=len(corr_matrix_df[0]))
pca.fit(corr_matrix_df)

PCA(n_components=181)

In [31]:
# print(pca.explained_variance_) # eigenvalues
print(pca.explained_variance_ratio_[0]) # Percentage of variance explained by each of the selected components.
print(pca.explained_variance_ratio_[1]) # Percentage of variance explained by each of the selected components.
print(pca.explained_variance_ratio_[2]) # Percentage of variance explained by each of the selected components.
# print(pca.components_[0])

0.5677010115555369
0.3935882624997912
0.015882634697186034


In [32]:
first_eigenvector = pca.components_[0]

for i in first_eigenvector:
    print(i)

0.05485063575111215
0.03745433777009233
-0.07511836897519486
-0.0767480182743303
-0.08522081845699142
-0.04879105430515436
-0.009354741773635869
-0.07753158393791171
-0.09222232560083538
-0.05908382236357833
0.02172223355464923
-0.09086660010174878
-0.10404717807497439
-0.09551712978563355
0.037268652131097535
-0.09411761203424068
-0.07296157910511293
-0.05983947640841351
-0.09129978566601718
-0.11127326349439876
-0.10311532342970521
-0.06041656207495619
-0.09655456551953244
-0.1047263711150052
-0.10527690486986042
-0.10258770213945251
-0.10618470617998288
-0.11026429457219587
-0.1054541593363287
-0.10905720711312443
-0.10890688893746954
-0.04663195802751239
0.023041308804984607
-0.025577660333113428
0.0009386961225700596
-0.041751482337057395
0.020967397571311348
0.0505070999744459
-0.05804102235256496
-0.06955337690835772
-0.046251228680354216
-0.07398158563686064
-0.0769956370910191
0.0158552035930194
-0.0936442860555347
-0.08805072197561878
-0.06960153204079339
-0.0
-0.0
-0.0532198